# NOAA Climate Normals Processing
Starter notebook for loading a single station's daily & hourly normals,
processing them into D3‑ready JSON structures.

In [222]:
import pandas as pd
import numpy as np
import json

# Adjust these filenames to point to your NOAA station CSVs
daily = pd.read_csv('daily_USW00012921.csv')
hourly = pd.read_csv('hourly_USW00012921.csv')

daily.head(), hourly.head()

(       STATION   DATE  LATITUDE  LONGITUDE  ELEVATION  \
 0  USW00012921  01-01   29.5442   -98.4839      240.5   
 1  USW00012921  01-02   29.5442   -98.4839      240.5   
 2  USW00012921  01-03   29.5442   -98.4839      240.5   
 3  USW00012921  01-04   29.5442   -98.4839      240.5   
 4  USW00012921  01-05   29.5442   -98.4839      240.5   
 
                          NAME  month  day  hour  DLY-TAVG-NORMAL  ...  \
 0  SAN ANTONIO INTL AP, TX US      1    1    99             51.8  ...   
 1  SAN ANTONIO INTL AP, TX US      1    2    99             51.8  ...   
 2  SAN ANTONIO INTL AP, TX US      1    3    99             51.7  ...   
 3  SAN ANTONIO INTL AP, TX US      1    4    99             51.7  ...   
 4  SAN ANTONIO INTL AP, TX US      1    5    99             51.7  ...   
 
   comp_flag_DLY-SNWD-25PCTL years_DLY-SNWD-25PCTL  DLY-SNWD-50PCTL  \
 0                                               0          -9999.0   
 1                                               0          -9

In [223]:

# Identify columns of interest if they exist
daily_cols = [
    c for c in daily.columns
    if any(k in c for k in [
        "TAVG", "TMAX", "TMIN", "DUTR", "PRCP", "SNOW", "SNWD"
    ])
]

print("\nColumns found:")
print(daily_cols)




Columns found:
['DLY-TAVG-NORMAL', 'meas_flag_DLY-TAVG-NORMAL', 'comp_flag_DLY-TAVG-NORMAL', 'years_DLY-TAVG-NORMAL', 'DLY-TMAX-NORMAL', 'meas_flag_DLY-TMAX-NORMAL', 'comp_flag_DLY-TMAX-NORMAL', 'years_DLY-TMAX-NORMAL', 'DLY-TMIN-NORMAL', 'meas_flag_DLY-TMIN-NORMAL', 'comp_flag_DLY-TMIN-NORMAL', 'years_DLY-TMIN-NORMAL', 'DLY-DUTR-NORMAL', 'meas_flag_DLY-DUTR-NORMAL', 'comp_flag_DLY-DUTR-NORMAL', 'years_DLY-DUTR-NORMAL', 'DLY-TAVG-STDDEV', 'meas_flag_DLY-TAVG-STDDEV', 'comp_flag_DLY-TAVG-STDDEV', 'years_DLY-TAVG-STDDEV', 'DLY-TMAX-STDDEV', 'meas_flag_DLY-TMAX-STDDEV', 'comp_flag_DLY-TMAX-STDDEV', 'years_DLY-TMAX-STDDEV', 'DLY-TMIN-STDDEV', 'meas_flag_DLY-TMIN-STDDEV', 'comp_flag_DLY-TMIN-STDDEV', 'years_DLY-TMIN-STDDEV', 'DLY-DUTR-STDDEV', 'meas_flag_DLY-DUTR-STDDEV', 'comp_flag_DLY-DUTR-STDDEV', 'years_DLY-DUTR-STDDEV', 'DLY-PRCP-PCTALL-GE001HI', 'meas_flag_DLY-PRCP-PCTALL-GE001HI', 'comp_flag_DLY-PRCP-PCTALL-GE001HI', 'years_DLY-PRCP-PCTALL-GE001HI', 'DLY-PRCP-PCTALL-GE010HI', 'meas_

In [224]:
# Show a *sample* of 10 random rows
print("\nSample (10 rows):")
print(daily[daily_cols].sample(10, random_state=1))


Sample (10 rows):
     DLY-TAVG-NORMAL meas_flag_DLY-TAVG-NORMAL comp_flag_DLY-TAVG-NORMAL  \
247             83.4                                                   R   
127             74.4                                                   R   
230             86.8                                                   R   
162             83.3                                                   R   
159             82.9                                                   R   
296             69.5                                                   R   
208             86.5                                                   R   
146             80.2                                                   R   
277             75.4                                                   R   
5               51.7                                                   R   

     years_DLY-TAVG-NORMAL  DLY-TMAX-NORMAL meas_flag_DLY-TMAX-NORMAL  \
247                     12             93.4                            

In [225]:
hourly_cols = [
    c for c in hourly.columns
    if any(k in c for k in [
        "TEMP", "DEWP", "HIDX", "WCHL",
        "CLOD", "WIND", "PRES"  # cloud cover, wind, pressure
    ])
]

print("\nColumns found:")
print(hourly_cols)




Columns found:
['HLY-TEMP-NORMAL', 'meas_flag_HLY-TEMP-NORMAL', 'comp_flag_HLY-TEMP-NORMAL', 'years_HLY-TEMP-NORMAL', 'HLY-TEMP-10PCTL', 'meas_flag_HLY-TEMP-10PCTL', 'comp_flag_HLY-TEMP-10PCTL', 'years_HLY-TEMP-10PCTL', 'HLY-TEMP-90PCTL', 'meas_flag_HLY-TEMP-90PCTL', 'comp_flag_HLY-TEMP-90PCTL', 'years_HLY-TEMP-90PCTL', 'HLY-DEWP-NORMAL', 'meas_flag_HLY-DEWP-NORMAL', 'comp_flag_HLY-DEWP-NORMAL', 'years_HLY-DEWP-NORMAL', 'HLY-DEWP-10PCTL', 'meas_flag_HLY-DEWP-10PCTL', 'comp_flag_HLY-DEWP-10PCTL', 'years_HLY-DEWP-10PCTL', 'HLY-DEWP-90PCTL', 'meas_flag_HLY-DEWP-90PCTL', 'comp_flag_HLY-DEWP-90PCTL', 'years_HLY-DEWP-90PCTL', 'HLY-PRES-NORMAL', 'meas_flag_HLY-PRES-NORMAL', 'comp_flag_HLY-PRES-NORMAL', 'years_HLY-PRES-NORMAL', 'HLY-PRES-10PCTL', 'meas_flag_HLY-PRES-10PCTL', 'comp_flag_HLY-PRES-10PCTL', 'years_HLY-PRES-10PCTL', 'HLY-PRES-90PCTL', 'meas_flag_HLY-PRES-90PCTL', 'comp_flag_HLY-PRES-90PCTL', 'years_HLY-PRES-90PCTL', 'HLY-CLOD-PCTCLR', 'meas_flag_HLY-CLOD-PCTCLR', 'comp_flag_HLY-CL

In [226]:
print("\nSample (10 rows):")
print(hourly[hourly_cols].sample(10, random_state=1))


Sample (10 rows):
      HLY-TEMP-NORMAL meas_flag_HLY-TEMP-NORMAL comp_flag_HLY-TEMP-NORMAL  \
4136             79.4                                                   R   
6705             73.9                                                   R   
3538             80.3                                                   R   
6583             68.8                                                   R   
1993             61.7                                                   R   
6751             67.4                                                   R   
6483             70.7                                                   R   
6951             81.5                                                   R   
4406             91.4                                                   R   
2021             59.8                                                   R   

      years_HLY-TEMP-NORMAL  HLY-TEMP-10PCTL meas_flag_HLY-TEMP-10PCTL  \
4136                     15             77.0               

## Compute Hourly Temperature Matrix
Pivot 365×24 hourly temps into D3‑friendly structure.

In [227]:
# Only keep rows where day != 99 and hour != 99
hourly = hourly[(hourly["day"] != 99) & (hourly["hour"] != 99)]

needed_cols = [
    "month", "day", "hour",
    "HLY-TEMP-NORMAL"
]

df = hourly[needed_cols].copy()

# Compute day-of-year
df["doy"] = pd.to_datetime(
    dict(year=2001, month=df.month, day=df.day)
).dt.dayofyear

# Sort for predictable ordering
df = df.sort_values(["doy", "hour"])

print(df.sample(20))
print(len(df), "rows")

      month  day  hour  HLY-TEMP-NORMAL  doy
5869      9    2    13             90.8  245
1678      3   11    22             62.0   70
4740      7   17    12             90.2  198
1494      3    4     6             53.1   63
693       1   29    21             54.2   29
6396      9   24    12             84.4  267
1960      3   23    16             75.7   82
1299      2   24     3             53.3   55
6674     10    6     2             69.9  279
2166      4    1     6             60.2   91
6587     10    2    11             80.7  275
6794     10   11     2             68.6  284
159       1    7    15             60.9    7
5420      8   14    20             88.5  226
5678      8   25    14             93.8  237
5279      8    8    23             82.9  220
7797     11   21    21             59.3  325
1357      2   26    13             66.0   57
5552      8   20     8             80.4  232
6063      9   10    15             89.6  253
8760 rows


In [228]:
df_out = df[["doy", "hour", "HLY-TEMP-NORMAL"]].rename(
    columns={"HLY-TEMP-NORMAL": "temp"}
)
df_out.to_csv("hourly_temp.csv", index=False)


## Daily High / Low Temperatures
Derived from hourly temps or daily normals.

In [210]:
# Numeric fields we care about
temp_cols = [
    "DLY-TAVG-NORMAL",
    "DLY-TMAX-NORMAL",
    "DLY-TMIN-NORMAL",
]

# Convert numeric & clean -9999
for col in temp_cols:
    daily[col] = pd.to_numeric(daily[col], errors="coerce")
    daily.loc[daily[col] < -100, col] = np.nan

# Convert month/day
daily["month"] = daily["month"].astype(int)
daily["day"]   = daily["day"].astype(int)

# Fake reference year (non-leap)
daily["date"] = pd.to_datetime(
    dict(year=2001, month=daily.month, day=daily.day),
    errors="coerce"
)

daily["doy"] = daily["date"].dt.dayofyear

# Final cleaned table
df_daily_clean = daily[
    ["doy", "month", "day", "date", "DLY-TAVG-NORMAL", "DLY-TMAX-NORMAL", "DLY-TMIN-NORMAL"]
].sort_values("doy").reset_index(drop=True)

df_daily_clean.sample(12)

,doy,month,day,date,DLY-TAVG-NORMAL,DLY-TMAX-NORMAL,DLY-TMIN-NORMAL
51,52.0,2,21,2001-02-21,58.3,69.6,47.0
120,121.0,5,1,2001-05-01,72.8,83.6,62.1
77,78.0,3,19,2001-03-19,65.1,76.0,54.1
257,258.0,9,15,2001-09-15,80.6,90.4,70.7
213,214.0,8,2,2001-08-02,87.0,97.6,76.5
131,132.0,5,12,2001-05-12,75.9,86.3,65.5
178,179.0,6,28,2001-06-28,84.1,93.9,74.3
145,146.0,5,26,2001-05-26,80.2,90.3,70.0
116,117.0,4,27,2001-04-27,72.0,82.8,61.1
156,157.0,6,6,2001-06-06,82.6,92.7,72.6


## Cloud Cover Categories by Day of Year
Categorize into clear / partly / cloudy and compute daily percentages.

In [211]:

# Cloud cover columns
cloud_cols = [
    "HLY-CLOD-PCTCLR",
    "HLY-CLOD-PCTFEW",
    "HLY-CLOD-PCTSCT",
    "HLY-CLOD-PCTBKN",
    "HLY-CLOD-PCTOVC",
]

# Convert month/day/hour
hourly["month"] = hourly["month"].astype(int)
hourly["day"]   = hourly["day"].astype(int)
hourly["hour"]  = hourly["hour"].astype(int)

# Convert cloud % values, fix -9999 → NaN, divide by 100 to get 0–1 proportions if desired
for col in cloud_cols:
    hourly[col] = pd.to_numeric(hourly[col], errors="coerce")
    hourly.loc[hourly[col] < -100, col] = np.nan

# Fake reference year (non-leap)
hourly["date"] = pd.to_datetime(
    dict(year=2001, month=hourly.month, day=hourly.day),
    errors="coerce"
)

hourly["doy"] = hourly["date"].dt.dayofyear

# Output clean dataframe
df_clouds_clean = hourly[
    ["doy", "month", "day", "hour", "date"] +
    cloud_cols
].sort_values(["doy", "hour"]).reset_index(drop=True)

df_clouds_clean.sample(12)


,doy,month,day,hour,date,HLY-CLOD-PCTCLR,HLY-CLOD-PCTFEW,HLY-CLOD-PCTSCT,HLY-CLOD-PCTBKN,HLY-CLOD-PCTOVC
8317,347,12,13,13,2001-12-13,NaN,NaN,NaN,NaN,NaN
7213,301,10,28,13,2001-10-28,NaN,NaN,NaN,NaN,NaN
7143,298,10,25,15,2001-10-25,24.1,25.9,17.9,21.4,10.7
6777,283,10,10,9,2001-10-10,15.1,17.3,9.8,28.4,29.3
1596,67,3,8,12,2001-03-08,10.3,18.8,10.3,21.1,39.5
2750,115,4,25,14,2001-04-25,NaN,NaN,NaN,NaN,NaN
7184,300,10,27,8,2001-10-27,NaN,NaN,NaN,NaN,NaN
3570,149,5,29,18,2001-05-29,4.0,27.1,24.4,34.7,9.8
7689,321,11,17,9,2001-11-17,20.6,13.9,2.7,15.2,47.5
1087,46,2,15,7,2001-02-15,NaN,NaN,NaN,NaN,NaN


In [212]:
# Ensure month/day/hour are integers
hourly["month"] = pd.to_numeric(hourly["month"], errors="coerce")
hourly["day"]   = pd.to_numeric(hourly["day"], errors="coerce")
hourly["hour"]  = pd.to_numeric(hourly["hour"], errors="coerce")

# Clean cloud % values (handle -9999)
for col in cloud_cols:
    hourly[col] = pd.to_numeric(hourly[col], errors="coerce")
    hourly.loc[hourly[col] < -100, col] = np.nan

# Add fake year → compute DOY
hourly["date"] = pd.to_datetime(
    dict(year=2001, month=hourly.month, day=hourly.day),
    errors="coerce"
)
hourly["doy"] = hourly["date"].dt.dayofyear

# --- Aggregate per day: average the hourly distributions ---
daily_cloud_distribution = (
    hourly.groupby("doy")[cloud_cols]
    .mean()
    .reset_index()
    .sort_values("doy")
)

daily_cloud_distribution.sample(12)


,doy,HLY-CLOD-PCTCLR,HLY-CLOD-PCTFEW,HLY-CLOD-PCTSCT,HLY-CLOD-PCTBKN,HLY-CLOD-PCTOVC
344,345,20.062500,17.550000,6.825000,14.175000,41.375000
49,50,20.050000,15.625000,7.725000,16.437500,40.137500
157,158,11.635714,30.371429,24.221429,22.792857,11.000000
150,151,8.125000,19.575000,16.462500,31.025000,24.825000
156,157,11.761538,28.923077,23.092308,24.315385,11.900000
173,174,5.925000,17.250000,18.975000,35.900000,21.962500
236,237,8.954545,27.163636,26.400000,27.181818,10.281818
176,177,6.000000,17.750000,18.612500,34.787500,22.862500
68,69,15.200000,12.625000,7.137500,22.350000,42.687500
262,263,13.075000,19.525000,16.562500,30.037500,20.812500


## Daily Chance of Precipitation


In [213]:


# Columns
col_precip_any = "DLY-PRCP-PCTALL-GE001HI"   # chance of ≥0.01" precip
col_snow_any   = "DLY-SNOW-PCTALL-GE001TI"   # chance of ≥0.1" snowfall

# Clean numeric values
for col in [col_precip_any, col_snow_any]:
    daily[col] = pd.to_numeric(daily[col], errors="coerce")
    daily.loc[daily[col] < -100, col] = np.nan

# Add DOY
daily["month"] = pd.to_numeric(daily["month"])
daily["day"]   = pd.to_numeric(daily["day"])

daily["date"] = pd.to_datetime(
    dict(year=2001, month=daily.month, day=daily.day),
    errors="coerce"
)
daily["doy"] = daily["date"].dt.dayofyear

# Probability components
p_any   = daily[col_precip_any]
p_snow  = daily[col_snow_any]
p_rain  = (p_any - p_snow).clip(lower=0)       # precip but no snow
p_mixed = p_any - p_rain - p_snow              # overlap area
p_mixed = p_mixed.clip(lower=0)                # no negatives

df_precip_types = pd.DataFrame({
    "doy": daily["doy"],
    "p_rain":  p_rain,
    "p_snow":  p_snow,
    "p_mixed": p_mixed,
    "p_any":   p_any,
}).sort_values("doy").reset_index(drop=True)

df_precip_types.sample(12)


,doy,p_rain,p_snow,p_mixed,p_any
149,150.0,23.8,0.0,0.000000e+00,23.8
241,242.0,19.0,0.0,0.000000e+00,19.0
328,329.0,19.3,0.0,0.000000e+00,19.3
4,5.0,21.1,0.1,1.415534e-15,21.2
63,64.0,24.3,0.0,0.000000e+00,24.3
156,157.0,21.7,0.0,0.000000e+00,21.7
364,365.0,21.6,0.1,1.415534e-15,21.7
181,182.0,20.3,0.0,0.000000e+00,20.3
195,196.0,17.1,0.0,0.000000e+00,17.1
272,273.0,22.2,0.0,0.000000e+00,22.2


## Sliding 30‑day Rainfall Sum


In [214]:


df = daily.copy()

# Fix missing values
for col in ["DLY-PRCP-PCTALL-GE001HI", "DLY-PRCP-50PCTL"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")
    df.loc[df[col] < -100, col] = np.nan

# Convert probability % → fraction
df["p_rain"] = df["DLY-PRCP-PCTALL-GE001HI"] / 100.0

# Expected daily rainfall inches
df["expected_rain"] = df["p_rain"] * df["DLY-PRCP-50PCTL"]

# Sort by day-of-year to ensure correct order
df = df.sort_values("doy")

# 31-day sliding rainfall (WeatherSpark method)
df["rain_31day"] = (
    df["expected_rain"]
    .rolling(window=31, center=True, min_periods=15)
    .sum()
)

df[["month", "day", "doy", "expected_rain", "rain_31day"]].sample(30)


,month,day,doy,expected_rain,rain_31day
303,10,30,303.0,0.02562,0.76380
342,12,8,342.0,0.01498,0.52016
218,8,6,218.0,0.01806,0.65387
126,5,6,126.0,0.04403,1.34890
277,10,4,277.0,0.03604,1.19458
129,5,9,129.0,0.04734,1.38693
308,11,4,308.0,0.02196,0.71673
304,10,31,304.0,0.02379,0.75379
134,5,14,134.0,0.05016,1.44852
1,1,2,2.0,0.02580,0.44392


In [215]:
import pandas as pd
import numpy as np

# --- ASSUMPTION ---
# Assuming your raw NCEI data is loaded into a DataFrame named 'daily'

# 1. Column Cleanup and Conversion
# FIX: Strip leading/trailing whitespace from all column names
daily.columns = daily.columns.str.strip()

# Clean and convert MTD-PRCP-NORMAL to a numeric type
# This is the cumulative sum of rain from the 1st of the month.
daily["MTD-PRCP-NORMAL"] = pd.to_numeric(daily["MTD-PRCP-NORMAL"], errors="coerce")
daily.loc[daily["MTD-PRCP-NORMAL"] < 0, "MTD-PRCP-NORMAL"] = np.nan # Remove -9999 placeholders

# 2. Date Setup
# Convert month/day columns into a sortable date column (using a dummy non-leap year)
daily["date"] = pd.to_datetime(
    dict(year=2001, month=daily["month"].astype(int), day=daily["day"].astype(int)),
    errors="coerce"
)
# Ensure the data is sorted correctly by station and then by date
daily = daily.sort_values(["STATION", "date"]) 

# Add day-of-year for easier analysis
daily["doy"] = daily["date"].dt.dayofyear

# 3. Calculate Daily Normal Precipitation (The Fix)
# The 'typical rainfall' is the difference between today's MTD and yesterday's MTD.
# We use groupby("STATION") to ensure the diff() calculation resets for each station.
daily["expected_rain"] = daily.groupby("STATION")["MTD-PRCP-NORMAL"].diff()

# The first day of any month (day == 1) has no 'previous day' within the MTD range.
# For day 1, MTD-PRCP-NORMAL IS the daily normal. We fill the resulting NaN 
# from the diff() operation with the actual MTD value.
daily.loc[daily["day"].astype(int) == 1, "expected_rain"] = daily["MTD-PRCP-NORMAL"]

# 4. Calculate Rolling Sum (Smoothing)
# Calculate the 31-day rolling sum (centered) to smooth the data, as requested earlier
daily["rain_31day"] = (
    daily["expected_rain"]
    .rolling(window=31, center=True, min_periods=1)
    .sum()
)

# 5. Final Output Table
# The 'expected_rain' column is your daily normal precipitation in inches.
df_precip_31day = daily[["doy", "month", "day", "expected_rain", "rain_31day"]]

# Display the final DataFrame
print(df_precip_31day.sample(30))


       doy  month  day  expected_rain  rain_31day
58    59.0      2   28           0.06        1.86
61    61.0      3    2           0.07        1.93
67    67.0      3    8           0.07        2.18
351  351.0     12   17           0.06        1.65
108  108.0      4   18           0.08        2.89
18    19.0      1   19           0.06        2.03
33    34.0      2    3           0.06        1.80
300  300.0     10   27           0.10        2.56
306  306.0     11    2           0.06        2.24
314  314.0     11   10           0.04        1.84
12    13.0      1   13           0.06        1.89
252  252.0      9    9           0.15        4.46
304  304.0     10   31           0.08        2.36
131  131.0      5   11           0.17        4.61
288  288.0     10   15           0.09        3.12
264  264.0      9   21           0.17        4.58
317  317.0     11   13           0.05        1.71
75    75.0      3   16           0.09        2.33
194  194.0      7   13           0.08        2.48


In [216]:


# 1. Column Cleanup and Conversion
daily.columns = daily.columns.str.strip() # FIX: Strip whitespace

# Clean and convert MTD columns to numeric, handling missing data (-9999)
daily["MTD-PRCP-NORMAL"] = pd.to_numeric(daily["MTD-PRCP-NORMAL"], errors="coerce").replace(-9999, np.nan)
daily["MTD-SNOW-NORMAL"] = pd.to_numeric(daily["MTD-SNOW-NORMAL"], errors="coerce").replace(-9999, np.nan)


# 2. Date Setup
daily["date"] = pd.to_datetime(
    dict(year=2001, month=daily["month"].astype(int), day=daily["day"].astype(int)),
    errors="coerce"
)
daily = daily.sort_values(["STATION", "date"]) 
daily["doy"] = daily["date"].dt.dayofyear


# 3. Calculate Daily Normals using MTD Difference

def calculate_daily_normal(df, mtd_column):
    """Calculates daily normal by taking the difference of MTD values."""
    
    # Calculate difference, grouped by station
    daily_normal = df.groupby("STATION")[mtd_column].diff()
    
    # Fill day 1 of the month (where diff is NaN) with the MTD value itself
    daily_normal.loc[df["day"].astype(int) == 1] = df.loc[df["day"].astype(int) == 1, mtd_column]
    
    return daily_normal

# Calculate Daily Total Precipitation (Rain + Melted Snow)
daily["daily_total_prcp"] = calculate_daily_normal(daily, "MTD-PRCP-NORMAL")

# Calculate Daily Snowfall (in inches of snow)
daily["daily_snowfall"] = calculate_daily_normal(daily, "MTD-SNOW-NORMAL")


# 4. Calculate Daily Rain-Only (Liquid Equivalent)

# NCEI's 'PRCP' is Total Liquid Equivalent. To get rain-only, we need to know the melted snow (liquid equivalent of snow).
# The standard assumption for melted snow is 10:1 (10 inches of snow = 1 inch of liquid). 
# However, NCEI's PRCP is the MEASURED liquid equivalent, so we use the SNOWFALL column only if the liquid equivalent is explicitly missing.

# IMPORTANT: The daily total precipitation ('daily_total_prcp') already accounts for melted snow.
# To find the 'rain-only' value, we must determine the liquid-equivalent of the snowfall. 
# Since this liquid-equivalent column (e.g., DLY-SNWD-LWE) is often missing,
# we cannot accurately split rain from melted snow using only the totals.

# ASSUMPTION for San Antonio (since it gets almost no snow): The daily_total_prcp is almost entirely rain.
# For stations where snow is common, you would need the DLY-SNOW-WE (Snow Water Equivalent) column, which is often missing 
# and not provided in the list of headers you gave. 

# If your goal is the TOTAL LIQUID precipitation (rain + melted snow), use 'daily_total_prcp'.
# If you must separate, the safest method is to use a 10:1 ratio assumption for melted snow:
daily["melted_snow_equiv"] = daily["daily_snowfall"] / 10.0
daily["daily_rain_only"] = daily["daily_total_prcp"] - daily["melted_snow_equiv"]
# Ensure rain-only isn't negative due to rounding/calculation errors, set min to 0.
daily.loc[daily["daily_rain_only"] < 0, "daily_rain_only"] = 0


# 5. Final Output and Rolling Sums
daily["rain_31day_total_prcp"] = (
    daily["daily_total_prcp"]
    .rolling(window=31, center=True, min_periods=1)
    .sum()
)

daily["rain_31day_rain_prcp"] = (
    daily["daily_rain_only"]
    .rolling(window=31, center=True, min_periods=1)
    .sum()
)

daily["rain_31day_snowprcp"] = (
    daily["melted_snow_equiv"]
    .rolling(window=31, center=True, min_periods=1)
    .sum()
)

df_split = daily[["doy", "month", "day", "rain_31day_total_prcp", "rain_31day_rain_prcp", "rain_31day_snowprcp"]]

print(df_split.sample(30))

       doy  month  day  rain_31day_total_prcp  rain_31day_rain_prcp  \
170  170.0      6   19                   2.64                  2.64   
302  302.0     10   29                   2.46                  2.46   
119  119.0      4   29                   3.57                  3.57   
100  100.0      4   10                   2.55                  2.55   
318  318.0     11   14                   1.67                  1.67   
293  293.0     10   20                   2.90                  2.90   
216  216.0      8    4                   1.63                  1.63   
35    36.0      2    5                   1.75                  1.75   
252  252.0      9    9                   4.46                  4.46   
271  271.0      9   28                   4.23                  4.23   
42    43.0      2   12                   1.64                  1.64   
172  172.0      6   21                   2.63                  2.63   
194  194.0      7   13                   2.48                  2.48   
154  1

In [220]:

# --- assume `hourly` is already loaded and cleaned ---

# use dew point normals
hourly["dewpt"] = pd.to_numeric(hourly["HLY-DEWP-NORMAL"], errors="coerce")
hourly.loc[hourly["dewpt"] < -100, "dewpt"] = np.nan

# build date + doy
hourly["date"] = pd.to_datetime(
    dict(year=2001,
         month=hourly["month"].astype(int),
         day=hourly["day"].astype(int)),
    errors="coerce"
)
hourly["doy"] = hourly["date"].dt.dayofyear

# --- classify dew point using WeatherSpark thresholds ---
def humidity_level(dp):
    if pd.isna(dp): return np.nan
    if dp < 55: return "dry"
    if dp < 60: return "comfortable"
    if dp < 65: return "humid"
    if dp < 70: return "muggy"
    if dp < 75: return "oppressive"
    return "miserable"

hourly["humidity_class"] = hourly["dewpt"].apply(humidity_level)

# --- daily percentages (WeatherSpark’s big chart) ---
daily_humidity = (
    hourly
    .groupby("doy")["humidity_class"]
    .value_counts(normalize=True)
    .rename("pct")
    .reset_index()
)

# pivot so each day has columns for each category
daily_humidity_pivot = daily_humidity.pivot(
    index="doy",
    columns="humidity_class",
    values="pct"
).fillna(0)

expected_cols = ["dry", "comfortable", "humid", "muggy", "oppressive", "miserable"]

# Ensure all columns exist
for col in expected_cols:
    if col not in daily_humidity_pivot.columns:
        daily_humidity_pivot[col] = 0.0



# what WeatherSpark labels "muggy, oppressive, or miserable"
daily_humidity_pivot["pct_muggy_plus"] = (
    daily_humidity_pivot[["muggy", "oppressive", "miserable"]].sum(axis=1)
)

daily_humidity_pivot.sample(50)


humidity_class,comfortable,dry,humid,muggy,oppressive,miserable,pct_muggy_plus
doy,,,,,,,
317,0.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.000000
259,0.000000,0.000000,0.250000,0.750000,0.000000,0.0,0.750000
157,0.000000,0.000000,0.000000,1.000000,0.000000,0.0,1.000000
284,0.333333,0.000000,0.666667,0.000000,0.000000,0.0,0.000000
345,0.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.000000
218,0.000000,0.000000,0.125000,0.375000,0.500000,0.0,0.875000
208,0.000000,0.000000,0.125000,0.416667,0.458333,0.0,0.875000
361,0.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.000000
51,0.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.000000


In [230]:
import math
from datetime import datetime, timedelta
import pandas as pd

# ---- Station Info ----
lat = math.radians(29.5442)
lon = -98.4839  # degrees
timezone_offset = -6  # US Central Standard Time in hours (no DST needed for normals)

# Equation helper
def solar_declination(day_of_year):
    return 0.409 * math.sin(2 * math.pi * day_of_year / 365 - 1.39)

def equation_of_time(day_of_year):
    B = 2 * math.pi * (day_of_year - 81) / 364
    return 9.87 * math.sin(2*B) - 7.53*math.cos(B) - 1.5*math.sin(B)

def hour_angle(lat, decl, altitude_deg):
    alt = math.radians(altitude_deg)
    h = (math.sin(alt) - math.sin(lat)*math.sin(decl)) / (math.cos(lat)*math.cos(decl))
    h = max(-1, min(1, h))  # clamp
    return math.acos(h)

def local_time_from_ha(ha, eqtime, lon):
    return 12 - (4*(math.degrees(ha)) + eqtime + lon*4) / 60

# Solar altitude thresholds
ALT_DAWN = -6        # civil dawn/dusk
ALT_SUNRISE =  -0.833  # sunrise/sunset (refraction + sun radius)

records = []

for doy in range(1, 366):

    decl = solar_declination(doy)
    eqt  = equation_of_time(doy)

    # Hour angles
    ha_sun = hour_angle(lat, decl, ALT_SUNRISE)
    ha_dawn = hour_angle(lat, decl, ALT_DAWN)

    # Convert to local times
    sunrise = local_time_from_ha( ha_sun, eqt, lon)
    sunset  = local_time_from_ha(-ha_sun, eqt, lon)

    dawn    = local_time_from_ha( ha_dawn, eqt, lon)
    dusk    = local_time_from_ha(-ha_dawn, eqt, lon)

    # Convert to actual datetimes
    base = datetime(2001, 1, 1) + timedelta(days=doy-1)
    t_dawn    = base + timedelta(hours=dawn + timezone_offset)
    t_sunrise = base + timedelta(hours=sunrise + timezone_offset)
    t_sunset  = base + timedelta(hours=sunset + timezone_offset)
    t_dusk    = base + timedelta(hours=dusk + timezone_offset)

    # Hour-by-hour classification
    for hr in range(24):
        t = datetime(2001, base.month, base.day, hr)
        if   t < t_dawn:    phase = "night"
        elif t < t_sunrise: phase = "twilight"
        elif t < t_sunset:  phase = "day"
        elif t < t_dusk:    phase = "twilight"
        else:               phase = "night"

        records.append({
            "doy": doy,
            "month": base.month,
            "day": base.day,
            "hour": hr,
            "phase": phase
        })

df_insolation = pd.DataFrame(records)
df_insolation.to_csv("insolation_grid.csv", index=False)

df_insolation.sample(20)


,doy,month,day,hour,phase
727,31,1,31,7,night
8703,363,12,29,15,day
6712,280,10,7,16,day
6420,268,9,25,12,day
7696,321,11,17,16,day
854,36,2,5,14,day
7568,316,11,12,8,day
3984,167,6,16,0,night
5357,224,8,12,5,night
8441,352,12,18,17,day


In [235]:
import sys
!"{sys.executable}" -m pip install astral


  Using cached astral-3.2-py3-none-any.whl.metadata (1.7 kB)
Using cached astral-3.2-py3-none-any.whl (38 kB)



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [236]:
from astral.sun import sun
from astral import LocationInfo
from datetime import datetime, timedelta
import pandas as pd

# ---- STATION INFO -------
lat = 29.5442
lon = -98.4839
tz = "US/Central"

loc = LocationInfo("Station", "USA", tz, lat, lon)

rows = []

# Non-leap reference year (WeatherSpark uses this approach)
year = 2001

for doy in range(1, 366):
    date = datetime(year, 1, 1) + timedelta(days=doy - 1)

    # Astral returns correct dawn/sunrise/sunset/dusk WITH DST applied
    s = sun(loc.observer, date=date, tzinfo=loc.timezone)
    dawn    = s["dawn"]
    sunrise = s["sunrise"]
    sunset  = s["sunset"]
    dusk    = s["dusk"]

    # Loop through a synthetic 24-hour day
    for hour in range(24):
        # Synthetic local-hour timestamp
        t = dawn.replace(hour=hour, minute=0, second=0, microsecond=0)

        # --- classification logic ---
        if t < dawn:
            phase = "night"
        elif dawn <= t < sunrise:
            phase = "twilight"
        elif sunrise <= t < sunset:
            phase = "day"
        elif sunset <= t < dusk:
            phase = "twilight"
        else:
            phase = "night"

        rows.append({
            "doy": doy,
            "month": date.month,
            "day": date.day,
            "hour": hour,
            "phase": phase,
            "datetime": t.isoformat()
        })

df_insolation = pd.DataFrame(rows)

# Save for D3
df_insolation.to_csv("insolation_grid.csv", index=False)

df_insolation.sample(20)


,doy,month,day,hour,phase,datetime
8628,360,12,26,12,day,2001-12-26T12:00:00-06:00
1646,69,3,10,14,day,2001-03-10T14:00:00-06:00
6614,276,10,3,14,day,2001-10-03T14:00:00-05:00
4127,172,6,21,23,night,2001-06-21T23:00:00-05:00
4617,193,7,12,9,day,2001-07-12T09:00:00-05:00
2918,122,5,2,14,day,2001-05-02T14:00:00-05:00
7156,299,10,26,4,night,2001-10-26T04:00:00-05:00
2844,119,4,29,12,day,2001-04-29T12:00:00-05:00
1962,82,3,23,18,day,2001-03-23T18:00:00-06:00
6847,286,10,13,7,night,2001-10-13T07:00:00-05:00
